# Import Section
---

In [1]:
import tensorflow as tf
import regex as re
import shutil
import json
import os

from object_detection.utils import label_map_util

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from ipywidgets import AppLayout, Button, Layout, Box, FloatText, Textarea, Dropdown, Label, IntSlider
from IPython.display import display, HTML
from IPython.display import clear_output
from ipyfilechooser import FileChooser

# Configure the Training Pipeline
---

In [2]:
class config_pipeline():
    
    def __init__(self, training_dir, my_model_directory_name, fine_tune_checkpoint,
                record_folder, label_map_pbtxt_fname, batch_size, num_steps, image_resizer, use_pretrain):
        self.training_dir = training_dir
        self.my_model_directory_name = my_model_directory_name
        self.fine_tune_checkpoint = fine_tune_checkpoint
        self.train_record_fname = record_folder + r'/train.record'
        self.test_record_fname  = record_folder + r'/test.record'
        self.label_map_pbtxt_fname = label_map_pbtxt_fname
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.image_resizer = image_resizer.split('X')[0]
        self.num_classes = 1
        self.use_pretrain = use_pretrain
        
        self.home_path = os.getcwd() 
        self.path_para_list = [self.my_model_directory_name, self.fine_tune_checkpoint, self.train_record_fname, self.test_record_fname, self.label_map_pbtxt_fname]
        self.update_path_para_list = list(map(lambda x : os.path.join(self.home_path, self.training_dir, x), self.path_para_list))  #update the
        
    def run_update(self):
        num_classes = self.get_num_classes(self.update_path_para_list[4])
        print('The number of class: {}'.format(num_classes))
        # create model_directory            
        self.create_user_folder(self.update_path_para_list[0])
        # copy pipeline.config
        self.update_config(self.update_path_para_list[1].split(r'checkpoint')[-2], self.update_path_para_list[0])
    
    def get_num_classes(self, pbtxt_fname):
        label_map = label_map_util.load_labelmap(pbtxt_fname)
        categories = label_map_util.convert_label_map_to_categories(
            label_map, max_num_classes=90, use_display_name=True)
        category_index = label_map_util.create_category_index(categories)
        return len(category_index.keys())
    
    def create_user_folder(self, dir_path):
        try:
            os.mkdir(dir_path)
        except OSError as error:
            print(error)
            print('skip create...')
    def copy_user_file(self, src, dst):
        try:
            shutil.copy(src, dst)
        except OSError as error:
            print(error)
    def update_config(self, src_fld, dst_fld):
        print('writing custom configuration file...')
    
        with open(os.path.join(src_fld, 'pipeline.config')) as f:
            s = f.read()
        print('The train config file is at: {}'.format(os.path.join(dst_fld, 'pipeline.config')))    
        with open(os.path.join(dst_fld, 'pipeline.config'), 'w') as f:
            
            # fine_tune_checkpoint
            if self.use_pretrain:
                s = re.sub('fine_tune_checkpoint: ".*?"',
                           'fine_tune_checkpoint: "{}"'.format(self.fine_tune_checkpoint), s)
            else:
                s = re.sub('fine_tune_checkpoint: ".*?"', '', s)
            # label_map_path
            s = re.sub(
                'label_map_path: ".*?"', 'label_map_path: "{}"'.format(self.label_map_pbtxt_fname), s)
            # Set training batch_size.
            s = re.sub('batch_size: [0-9]+',
                       'batch_size: {}'.format(self.batch_size), s)
            # Set training steps, num_steps
            s = re.sub('num_steps: [0-9]+',
                       'num_steps: {}'.format(self.num_steps), s)
            # Set number of classes num_classes.
            s = re.sub('num_classes: [0-9]+',
                       'num_classes: {}'.format(self.num_classes), s)
            # Set number of fixed_shape_resizer.
            s = re.sub('height: [0-9]+',
                       'height: {}'.format(self.image_resizer), s)
            s = re.sub('width: [0-9]+',
                       'width: {}'.format(self.image_resizer), s)
            #fine-tune checkpoint type
            s = re.sub(
                'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
            
            # tfrecord files train and test. (the train section must before test section)
            s = re.sub(
                '(input_path: ".*?)(PATH_TO_BE_CONFIGURED)(.*?")', 'input_path: "{}"'.format(self.train_record_fname), s, 1)
            s = re.sub(
                '(input_path: ".*?)(PATH_TO_BE_CONFIGURED)(.*?")', 'input_path: "{}"'.format(self.test_record_fname), s, 1)
            
            f.write(s)            

# Widgets Control Section
---

In [3]:

class train_config_and_cmds_widgets():
    def __init__(self):
        
        self.tflite_file_loc = ""
        
        form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between',
        )
        
        ### open source data download###
        self.A_ta = widgets.Text(value='training_demo', placeholder='Type something', disabled=False)
        self.B_ta = widgets.Text(value='models/my_ssd_mobilenet_v2_fpnlite', placeholder='Type something', disabled=False)
        self.J_ta = widgets.Checkbox(value=True, disabled=False, indent=False)
        self.C_ta = widgets.Textarea(value='pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0', 
                                     placeholder='Type something', disabled=False)
        self.D_ta = widgets.Text(value='annotations', placeholder='Type something', disabled=False)
        self.E_ta = widgets.Text(value='annotations/label_map.pbtxt', placeholder='Type something', disabled=False)
        self.F_ta = widgets.IntSlider(value=16, min=8, max=72, step=4)
        self.G_ta = widgets.Text(value='50000', placeholder='Type something', disabled=False)
        self.H_ta = Dropdown(options=['320X320', '240X240', '180X180', '120X120', '80X80', '60X60'])
        self.I_ta = widgets.Text(value='exported-models/tflite_infer_graph', placeholder='Type something', disabled=False)
        
        form_pretrain_itesms = [Box([Label(value = 'Use pre-train model'), self.J_ta], layout=form_item_layout),
            Box([Label(value = 'Fine Tune Checkpoint'), self.C_ta], layout=form_item_layout)]
        
        form_train_items = [
            Box([Label(value = 'Your Working Directory Name'), self.A_ta], layout=form_item_layout),
            Box([Label(value = 'My Model Directory Name'), self.B_ta], layout=form_item_layout),
            Box(form_pretrain_itesms, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px lightblue',    
        )),
            Box([Label(value = 'Train Record Folder'), self.D_ta], layout=form_item_layout),
            Box([Label(value = 'Label Map Pbtxt'), self.E_ta], layout=form_item_layout),
            Box([Label(value = 'Batch Size'), self.F_ta], layout=form_item_layout),
            Box([Label(value = 'Number Steps'), self.G_ta], layout=form_item_layout),
            Box([Label(value = 'Image Resizer'), self.H_ta], layout=form_item_layout),
            Box([Label(value = 'Output Folder'), self.I_ta], layout=form_item_layout)
        ]
        
        self.form_output_train_cmd = Box(form_train_items, layout=Layout(
            display='flex',
            flex_flow='column',
            border='solid 3px lightgreen',
            align_items='stretch',
            width='50%',
        ))
        
    def move_allfiles(self, src_folder, dst_folder):
        copy_num = 0
        
        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  #copy whole folder
                shutil.move(fullpath, dst_folder)
                print("Copy finish: {}".format(f))
    
    def show_headline(self, output):
        html0= widgets.HTML(value = f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)
    
    def show_main(self):   
        
        intro_text = 'Please Choose the setting of train config'
        htmlWidget = widgets.HTML(value = f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(htmlWidget)
        
        #Create an accordion and put the 2 boxes
        accordion = widgets.Accordion(children=[self.form_output_train_cmd 
                                                ]).add_class("parentstyle")
        #Add a custom style tag to the notebook, you can use dev tool to inspect the class names
        display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        accordion.set_title(0, 'Configure the Training')
        
        def act_para(training_dir, my_model_directory_name, fine_tune_checkpoint,
                     record_folder, label_map_pbtxt_fname, batch_size, 
                     num_steps, image_resizer, output_dir, use_pretrain):
            #------------------#
            # The main executing Toggle_Button
            #------------------#
            toggle_update_config = widgets.ToggleButton(description='Update the config', 
                                                   layout=Layout(width='30%', height='30px'), button_style='success')
            toggle_create_cmd = widgets.ToggleButton(description='Create The Commands', 
                                                   layout=Layout(width='30%', height='30px'), button_style='success')
            out = widgets.Output(layout=Layout(border = '1px solid green'))
            
            if use_pretrain:
                self.C_ta.layout.visibility = 'visible'
            else:
                self.C_ta.layout.visibility = 'hidden'  
            
            
            #------------------#
            # The main executing Toggle_Button's event function
            #------------------#
            def run_update_config(obj):
                with out:
                    if obj['new']:
                        out.clear_output()
                        print('Update training config...')
                        config_obj = config_pipeline(training_dir, my_model_directory_name, fine_tune_checkpoint,
                                                     record_folder, label_map_pbtxt_fname, batch_size, 
                                                     num_steps, image_resizer, use_pretrain)
                        config_obj.run_update()
                        
                        print('Finish')
                    else:
                        print('stop')
                        out.clear_output() 
            
            def run_create_cmd(obj):
                with out:
                    if obj['new']:
                        out.clear_output()
                        self.show_headline('Please copy these commands to the CMD.exe Prompt to execute. ')
                        pipeline_config_path = (my_model_directory_name + r'/pipeline.config')
                        checkpoint_dir = my_model_directory_name
                        output_directory = output_dir
                        try:
                            os.mkdir(os.path.join(os.getcwd(), training_dir, output_directory))
                        except OSError as error:
                            #print(error)
                            print('skip create...')
                        
                        self.show_headline('(1.) Please excute the below 2~5 commands under the working directory')
                        print("cd {}".format(os.path.join(os.getcwd(), training_dir)))
                        
                        self.show_headline('(2.) Train:')
                        train_cmd = "python model_main_tf2.py --model_dir={} --pipeline_config_path={}".format(my_model_directory_name, 
                                    pipeline_config_path)
                        print(train_cmd)
                        
                        self.show_headline('(3.) Evaluating the Model (Optional):')
                        evl_cmd = "python model_main_tf2.py --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(my_model_directory_name, 
                                    pipeline_config_path, checkpoint_dir)
                        print(evl_cmd)
                        
                        self.show_headline('(4.) Monitor Training:')
                        monitor_cmd = "tensorboard --logdir={}".format(my_model_directory_name)    
                        print(monitor_cmd)
                        
                        self.show_headline('(5.) Export a tflite inference graph:')
                        export_graph_cmd = "python export_tflite_graph_tf2.py --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={} ".format(pipeline_config_path, checkpoint_dir, output_directory)    
                        print(export_graph_cmd)
                        
                    else:
                        print('stop')
                        out.clear_output() 
            
            toggle_update_config.observe(run_update_config, 'value')
            toggle_create_cmd.observe(run_create_cmd, 'value')
            display(toggle_update_config, toggle_create_cmd)
            display(out)
            
        
        #------------------#
        # widgets.Accordion's interactive input with action function `act_para()`
        #------------------#
        out_inter = widgets.interactive_output(act_para, {'training_dir': self.A_ta, 'my_model_directory_name': self.B_ta, 'fine_tune_checkpoint': self.C_ta, 
                                                          'record_folder': self.D_ta, 'label_map_pbtxt_fname': self.E_ta, 'batch_size': self.F_ta, 
                                                          'num_steps': self.G_ta, 'image_resizer': self.H_ta, 
                                                          'output_dir': self.I_ta, 'use_pretrain': self.J_ta
                                                          })
        display(accordion, out_inter)
        

# Run Section
---
- The detail description of all the parameters and each step meaning is here [meaning](#id-train_evl_monitor)
- In this notebook step, you have alreay finish the dataset prepared. If not, please go to `image_dataset\create_data.ipynb`.

In [4]:
act = train_config_and_cmds_widgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Please Choose the setting of train config</b>")

Accordion(children=(Box(children=(Box(children=(Label(value='Your Working Directory Name'), Text(value='traini…

Output()

<a id="id-train_evl_monitor"></a>
# Parameter & Steps Description
---


## Training the Model
### Download Pre-Trained Model
- The model in this demo is the `SSD MobileNet V2 FPNLite 320x320` which is already in `pre-trained-models/` 
- All of the tensorflow2 pre-trained models are listed in [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md), and you can download each of them.
- The download file is `*.tar.gz`, and please decompression it (e.g. 7zip, WinZIP, etc.).
- Move the decompression folder into `<Your Woking Folder>/pre-trained-models`

- <pre>training_demo/
├─ ...
├─ pre-trained-models/
│  └─ ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
│     ├─ checkpoint/
│     ├─ saved_model/
│     └─ pipeline.config
└─ ...
</pre>
### Configure the Training
- The parameters below is basing on your files/folders nameing. Please update them if any change.
    1. `Your Working Directory Name`: The user defined working directory
    2. `My Model Directory Name`: The location of user defined which save the training's weights, checkpoints and config file
    3. `Fine Tune Checkpoint`: The location of downloaded pre-trained-models checkpoint
    4. `Train Record Folder`: The folder location of created tfrecord for training & testing
    6. `Label Map Pbtxt`: The file location of label map
    7. `Batch Size`: Tune this value depending on the available memory.
    8. `Number Steps`: How many the training steps.
    9. `Image Resizer`: The input layer of height & width. The smaller the accracy is lower but training/inference is faster. 
    10. `Output Folder`: The location of exporting the tflite inference graph. 
- This is for `SSD MobileNet V2 FPNLite 320x320` pipeline.config, if you use other model, the pipeline.config maybe have minor different. However, these attributes should be the same and mattered.
- \<Advanced>: If you want to tunning more parameters, please update `pipeline.config` directly.

### Train
- The output will normally look like it has “frozen”, but DO NOT rush to cancel the process. The training outputs logs only every 100 steps by default, therefore if you wait for a while, you should see a log for the loss at step 100.
- <img src="train_exmple_plots/train_process.png" width="400" height="300">

## Evaluating the Model (Optional)

## Monitor Training Job Progress using TensorBoard

## Export a TFLite inference graph